In [ ]:
import config as cfg
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from pathlib import Path
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
data = pd.read_pickle(cfg.PATHS.TRANSFORMED_DATA_PICKLE)
data.shape

In [ ]:
import keras.backend as K

def custom_loss(y_true, y_pred):
    # Compute the weight for each output based on the corresponding non-zero value of the predictor
    predictor_weight = K.cast(K.not_equal(y_true, 0), dtype='float32')
    
    # Compute the weighted mean squared error loss
    mse = K.mean(K.square(y_true - y_pred) * predictor_weight, axis=-1)
    
    return mse

In [ ]:
from keras.layers import Input, Dense, Concatenate, Flatten
from keras.models import Model

# Define the input layers
review_input = Input(shape=(embedding_dim,))
predictor_input = Input(shape=(num_predictors,))

# Concatenate the review text embedding and predictor inputs
concatenated = Concatenate()([review_input, predictor_input])

# Define the fully connected layers
fc0 = Dense(256, activation='relu')(concatenated)
fc1 = Dense(128, activation='relu')(fc0)
fc2 = Dense(64, activation='relu')(fc1)
fc3 = Dense(32, activation='relu')(fc2)

# Define the output layers
helpful_output = Dense(1, name='helpful')(fc3)
funny_output = Dense(1, name='funny')(fc3)
cool_output = Dense(1, name='cool')(fc3)

# Define the model with multiple outputs
model = Model(inputs=[review_input, predictor_input], outputs=[helpful_output, funny_output, cool_output])

# Compile the model
model.compile(optimizer='adam', loss=custom_loss, metrics=['mae'])
